In [3]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
%cd /gdrive/MyDrive/Code\ Repository/aidea/ANS/AIdea_ANS

/gdrive/MyDrive/Code Repository/aidea/ANS/AIdea_ANS


In [ ]:
!pip install -r requirements.txt

In [ ]:
from torch.cuda import is_available
print(is_available())

False


In [5]:
!python train.py

{'train_data_path': '/gdrive/MyDrive/Code Repository/aidea/ANS/data/train_classification', 'test_data_path': '/gdrive/MyDrive/Code Repository/aidea/ANS/data/test', 'saving_path': '/gdrive/MyDrive/Code Repository/aidea/ANS/data', 'class': 'train', 'train_per': 0.5, 'signal_bound': 185000, 'epochs': 1, 'batch_size': 32, 'lr': 0.0002, 'beta1': 0.5, 'beta2': 0.999}
train_num:	 832
device:	 cpu
version:	 8
epoch:  0
  0% 0/26 [00:01<?, ?it/s]
Traceback (most recent call last):
  File "train.py", line 118, in <module>
    main()
  File "train.py", line 72, in main
    for i, data in enumerate(progress_bar):
  File "/usr/local/lib/python3.7/dist-packages/tqdm/std.py", line 1180, in __iter__
    for obj in iterable:
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 521, in __next__
    data = self._next_data()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 561, in _next_data
    data = self._dataset_fetcher.fetch(index)  